In [136]:
# importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter

In [150]:
# loading the dataset and dropping one column, since we only need the text and sentiment
dataset = pd.read_csv('data/train.csv')
df = pd.DataFrame(data=dataset, columns=['text', 'sentiment'])
df = df.dropna()
#creating word count, uppercase and special char. word count
import string
df['Word Count'] = [len(review.split()) for review in df['text']]

df['Uppercase Char Count'] = [sum(char.isupper() for char in review) \
                              for review in df['text']]                           

df['Special Char Count'] = [sum(char in string.punctuation for char in review) \
                            for review in df['text']]   
# Formatting the datatypes
df['sentiment'] = df['sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0})

In [151]:
df.head(10)

,text,sentiment,Word Count,Uppercase Char Count,Special Char Count
0,Spent the entire morning in a meeting w/ a ven...,1,28,3,5
1,Oh! Good idea about putting them on ice cream,2,9,2,1
2,says good (or should i say bad?) afternoon! h...,1,9,0,10
3,i dont think you can vote anymore! i tried,0,9,0,1
4,haha better drunken tweeting you mean?,2,6,0,1
5,headache wanna see my Julie,0,5,1,0
6,had an awsome salad! I recommend getting the S...,2,12,2,2
7,fine! Going to do my big walk today 20 or so ...,2,12,1,1
8,Thank a yoou how are you? #TwitterTakeover,2,7,3,2
9,Why don't adobe realise no one WANTS to pay fo...,1,23,7,1


In [166]:
# here should be shown that the descriptive statistcis (mean, median wordcount etc.) 
# are the same for all three groups.

In [194]:
df['text']

0        Spent the entire morning in a meeting w/ a ven...
1            Oh! Good idea about putting them on ice cream
2        says good (or should i say bad?) afternoon!  h...
3               i dont think you can vote anymore! i tried
4                   haha better drunken tweeting you mean?
                               ...                        
27481    I want to go to VP, but no one is willing to c...
27482                                Wah, why are you sad?
27483    playing sudoku while mommy makes me breakfast ...
27484                   see u bye see u!  i love the hot30
27485           ha ha, and what game is that? i like games
Name: text, Length: 27485, dtype: object

In [195]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text'].apply(lambda x: [item for item in x if item not in stop])

[nltk_data] Downloading package stopwords to /Users/tom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [S, p, e, n,  , h, e,  , e, n, r, e,  , r, n, ...
1        [ , O, h, !,  , G,  , e,  , b, u,  , p, u, n, ...
2        [ , g,  , (, r,  , h, u, l,  ,  ,  , b, ?, ), ...
3        [ ,  , n,  , h, n, k,  , u,  , c, n,  , v, e, ...
4        [ , h, h,  , b, e, e, r,  , r, u, n, k, e, n, ...
                               ...                        
27481    [I,  , w, n,  ,  , g,  ,  , V, P, ,,  , b, u, ...
27482           [ , W, h, ,,  , w, h,  , r, e,  , u,  , ?]
27483    [p, l, n, g,  , u, k, u,  , w, h, l, e,  ,  , ...
27484    [ , e, e,  , u,  , b, e,  , e, e,  , u, !,  , ...
27485    [ ,  , h,  , h, ,,  , n,  , w, h,  , g, e,  , ...
Name: text, Length: 27485, dtype: object

In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
bow = vectorizer.fit_transform(df['text'])
sentiment = df['sentiment']

In [197]:
len(vectorizer.get_feature_names())

26447

In [212]:
vectorizer = TfidfVectorizer(min_df=30)
bow = vectorizer.fit_transform(df['text'])
len(vectorizer.get_feature_names())

1062

In [213]:
from sklearn.feature_selection import SelectKBest, chi2
# select the 500 features that have the strongest correlation to a class from the
# remaining 1481 features.
selected_features = \
SelectKBest(chi2, k=500).fit(bow, labels).get_support(indices=True)

In [214]:
vectorizer = TfidfVectorizer(min_df=30)
bow = vectorizer.fit_transform(df['text'])
bow

<27485x1062 sparse matrix of type '<class 'numpy.float64'>'
	with 248533 stored elements in Compressed Sparse Row format>

In [215]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bow, sentiment, test_size=0.25)

In [221]:
from sklearn.ensemble import RandomForestClassifier as rfc
classifier = rfc()
classifier.fit(X_train,y_train)
classifier.score(X_test,y_test)

0.6708381839348079

In [219]:
# BONUS: hyperparameter optimization
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

classifier = rfc()

hyperparameters = {
    'n_estimators':stats.randint(10,300),
    'criterion':['gini','entropy'],
    'min_samples_split':stats.randint(2,9),
    'bootstrap':[True,False]
}

random_search = RandomizedSearchCV(classifier, hyperparameters, n_iter=50, n_jobs=8)

random_search.fit(bow, sentiment)

KeyboardInterrupt: 

In [220]:
# Take the best of the 100 trained models
optimized_classifier = random_search.best_estimator_
optimized_classifier.fit(X_train,y_train)

optimized_classifier.score(X_test,y_test)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'